In [ ]:
from huggingface_hub import login

# Saving Models
Be careful not to overwrite!


In [ ]:
model_repo_name = "Benw8888/lp_saes"
model_id = "Benw8888/lp_saes"
mode = "bias"
chkpt = 0
local_file = f"/root/sparsify/notebooks/trained_models/sae_l0.1_0.1.pt"
file_in_repo = (
    f"sft_sae_70m.pt"  # "sft_sae_6b.pt" "transfer_sft_ppo_6b.pt" f"transfer_sft_base_6b_{mode}.pt"
)
# each checkpoint was 8mil tokens, so chkpt 6 was 56mil tokens (0 indexed)
from huggingface_hub import HfApi

api = HfApi()
api.upload_file(
    path_or_fileobj=local_file,
    path_in_repo=file_in_repo,
    repo_id=model_id,
    repo_type="model",
)

In [ ]:
from huggingface_hub import HfApi

l1_alphas = [
    1e-6,
    3e-6,
    1e-5,
    3e-5,
    1e-4,
    3e-4,
    1e-3,
    3e-3,
    1e-2,
    3e-2,
    1e-1,
    3e-1,
    1,
    3,
]
ps = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1, 1.1]

model_repo_name = "Benw8888/lp_saes"
model_id = "Benw8888/lp_saes"

folder = "toy_saes"
api = HfApi()
api.upload_folder(
    folder_path="/root/sparsify/trained_models/toy_saes",  # {folder}",
    path_in_repo=folder,
    repo_id=model_id,
    repo_type="model",
)


# Loading Models
Downloading Files


In [ ]:
model_repo_name = "Benw8888/pythia-6.9b-sae"
model_id = "Benw8888/pythia-6.9b-sae"
modes = ["scale", "rotation", "bias", "free"]
huggingface_model_names = []
# huggingface_model_names += ["rm_sae_gptj.pt", "ppo_sae_6b.pt"] + [f"transfer_rm_ppo_6b_{mode}.pt" for mode in modes]
# huggingface_model_names += ["base_sae_6b.pt", "sft_sae_6b.pt"] + [f"transfer_base_sft_6b_{mode}.pt" for mode in modes]
# huggingface_model_names += [f"transfer_sft_base_6b_{mode}_6.pt" for mode in modes]
# huggingface_model_names += [f"transfer_base_sft_6b_{mode}_6.pt" for mode in modes]
# huggingface_model_names += [f"{model}_dead_features.pt" for model in ["base","sft","rm","ppo"]]
huggingface_model_names += ["base_sae_70m.pt", "sft_sae_70m.pt"]
huggingface_model_names += [f"transfer_base_sft_70m_{mode}.pt" for mode in modes]
huggingface_model_names += [f"transfer_sft_base_70m_{mode}.pt" for mode in modes]


# huggingface_model_names = ["base_sae_6b.pt"]
from huggingface_hub import hf_hub_download
import torch
# model_id = "Elriggs/pythia-70m-deduped-layer-2"

for hf_name in huggingface_model_names:
    ae_download_location = hf_hub_download(repo_id=model_id, filename=hf_name)
    downloaded_autoencoder = torch.load(ae_download_location)

    import os

    if not os.path.exists("trained_models"):
        os.makedirs("trained_models")
    # Save model
    torch.save(downloaded_autoencoder, f"trained_models/{hf_name}")

Downloading From Folders


In [ ]:
model_repo_name = "Benw8888/pythia-6.9b-sae"
model_id = "Benw8888/pythia-6.9b-sae"
# modes = ["scale", "rotation", "bias", "free"]
huggingface_folder_names = []
huggingface_folder_names += [
    "base_frequency_70m",
    "base_dead_features_70m",
    "base_sae_70m",
    "base_retrain_70m",
]  # , "transfer_base_sft_70m"] "base_autoTED_70m"
layers = [0, 1, 2, 3, 4, 5]
l1s = [f"{l1_alpha}" for l1_alpha in [1e-3, 2e-3, 4e-3, 8e-3]]

from huggingface_hub import hf_hub_download
import torch
import os
# model_id = "Elriggs/pythia-70m-deduped-layer-2"

for folder in huggingface_folder_names:
    if not os.path.exists("trained_models"):
        os.makedirs("trained_models")
    if not os.path.exists(f"trained_models/{folder}"):
        os.makedirs(f"trained_models/{folder}")
    for layer in layers:
        for l1 in l1s:
            file_name = f"{folder}_{layer}_{l1}.pt"
            if folder == "base_retrain_70m":
                file_name = f"base_autoTED_70m_scale_{layer}_{l1}.pt"
            if folder[:8] == "transfer":
                file_name = f"{folder}_free_{layer}_{l1}.pt"
            hf_name = f"{folder}/{file_name}"
            ae_download_location = hf_hub_download(repo_id=model_id, filename=hf_name)
            downloaded_autoencoder = torch.load(ae_download_location)
            torch.save(downloaded_autoencoder, f"trained_models/{folder}/{file_name}")

Downloading reward model stuff??

In [ ]:
# save reward model

from huggingface_hub import hf_hub_download
import torch

ae_download_location = hf_hub_download(repo_id="Dahoas/gptj-rm-static", filename="hf_ckpt.pt")
downloaded_rm = torch.load(ae_download_location)

import os

if not os.path.exists("llms"):
    os.makedirs("llms")
# Save model
torch.save(downloaded_rm, f"llms/gptj-rm-static.pt")
downloaded_rm